In [0]:
%sql
SELECT 
  account_id,
  workspace_id,
  job_id,
  run_id,
  period_start_time,
  period_end_time,
  task_key,
  compute_ids,
  result_state,
  job_run_id,
  parent_run_id,
  CASE 
    WHEN result_state = 'SUCCESS' THEN 'SUCCESS'
    WHEN result_state IN ('ERROR', 'FAILED', 'SKIPPED') THEN 'ERROR'
    ELSE result_state
  END AS result_state,
  (unix_timestamp(period_end_time) - unix_timestamp(period_start_time)) / 60 AS duration_minutes
FROM system.lakeflow.job_task_run_timeline
WHERE period_start_time > (SELECT MIN(change_time) FROM system.lakeflow.jobs)
  AND job_id IN (
    SELECT DISTINCT job_id
    FROM system.lakeflow.jobs
    WHERE change_time > (SELECT MAX(delete_time) + INTERVAL 1 MINUTE FROM system.lakeflow.jobs)
  )
ORDER BY duration_minutes DESC